# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 18 2022 3:10PM,242096,10,8290190,Eywa Pharma Inc.,Released
1,Jul 18 2022 3:10PM,242096,10,8290191,Eywa Pharma Inc.,Released
2,Jul 18 2022 3:10PM,242096,10,8290193,Eywa Pharma Inc.,Released
3,Jul 18 2022 3:10PM,242096,10,8290194,Eywa Pharma Inc.,Released
4,Jul 18 2022 3:10PM,242096,10,8278843,Eywa Pharma Inc.,Released
5,Jul 18 2022 3:10PM,242096,10,8290150,Eywa Pharma Inc.,Released
6,Jul 18 2022 3:10PM,242096,10,8290151,Eywa Pharma Inc.,Released
7,Jul 18 2022 3:10PM,242096,10,8290152,Eywa Pharma Inc.,Released
8,Jul 18 2022 3:10PM,242096,10,8290153,Eywa Pharma Inc.,Released
9,Jul 18 2022 3:10PM,242096,10,8290154,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
56,242132,Released,1
57,242133,Released,1
58,242135,Released,1
59,242136,Released,1
60,242137,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242132,NaN,NaN,1.0
242133,NaN,NaN,1.0
242135,NaN,NaN,1.0
242136,NaN,NaN,1.0
242137,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241900,0.0,6.0,6.0
241972,1.0,8.0,5.0
241980,0.0,26.0,5.0
242005,0.0,0.0,3.0
242007,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241900,0,6,6
241972,1,8,5
241980,0,26,5
242005,0,0,3
242007,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241900,0,6,6
1,241972,1,8,5
2,241980,0,26,5
3,242005,0,0,3
4,242007,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241900,,6,6
1,241972,1,8,5
2,241980,,26,5
3,242005,,,3
4,242007,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 18 2022 3:10PM,242096,10,Eywa Pharma Inc.
13,Jul 18 2022 3:05PM,242137,21,"NBTY Global, Inc."
14,Jul 18 2022 3:03PM,242136,19,"GUSA Granules USA, Inc."
15,Jul 18 2022 3:01PM,242135,12,HealthCaps LLC
16,Jul 18 2022 2:49PM,242133,10,"ClearSpec, LLC"
17,Jul 18 2022 2:47PM,242132,10,"ClearSpec, LLC"
18,Jul 18 2022 2:47PM,242131,10,Emerginnova
21,Jul 18 2022 2:46PM,242130,10,"ClearSpec, LLC"
22,Jul 18 2022 2:38PM,242129,21,"NBTY Global, Inc."
23,Jul 18 2022 2:34PM,242128,19,ACG North America LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 18 2022 3:10PM,242096,10,Eywa Pharma Inc.,,,13
1,Jul 18 2022 3:05PM,242137,21,"NBTY Global, Inc.",,,1
2,Jul 18 2022 3:03PM,242136,19,"GUSA Granules USA, Inc.",,,1
3,Jul 18 2022 3:01PM,242135,12,HealthCaps LLC,,,1
4,Jul 18 2022 2:49PM,242133,10,"ClearSpec, LLC",,,1
5,Jul 18 2022 2:47PM,242132,10,"ClearSpec, LLC",,,1
6,Jul 18 2022 2:47PM,242131,10,Emerginnova,,,3
7,Jul 18 2022 2:46PM,242130,10,"ClearSpec, LLC",,,1
8,Jul 18 2022 2:38PM,242129,21,"NBTY Global, Inc.",,,1
9,Jul 18 2022 2:34PM,242128,19,ACG North America LLC,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 18 2022 3:10PM,242096,10,Eywa Pharma Inc.,13,,
1,Jul 18 2022 3:05PM,242137,21,"NBTY Global, Inc.",1,,
2,Jul 18 2022 3:03PM,242136,19,"GUSA Granules USA, Inc.",1,,
3,Jul 18 2022 3:01PM,242135,12,HealthCaps LLC,1,,
4,Jul 18 2022 2:49PM,242133,10,"ClearSpec, LLC",1,,
5,Jul 18 2022 2:47PM,242132,10,"ClearSpec, LLC",1,,
6,Jul 18 2022 2:47PM,242131,10,Emerginnova,3,,
7,Jul 18 2022 2:46PM,242130,10,"ClearSpec, LLC",1,,
8,Jul 18 2022 2:38PM,242129,21,"NBTY Global, Inc.",1,,
9,Jul 18 2022 2:34PM,242128,19,ACG North America LLC,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 18 2022 3:10PM,242096,10,Eywa Pharma Inc.,13,,
1,Jul 18 2022 3:05PM,242137,21,"NBTY Global, Inc.",1,,
2,Jul 18 2022 3:03PM,242136,19,"GUSA Granules USA, Inc.",1,,
3,Jul 18 2022 3:01PM,242135,12,HealthCaps LLC,1,,
4,Jul 18 2022 2:49PM,242133,10,"ClearSpec, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 18 2022 3:10PM,242096,10,Eywa Pharma Inc.,13.0,NaN,NaN
1,Jul 18 2022 3:05PM,242137,21,"NBTY Global, Inc.",1.0,NaN,NaN
2,Jul 18 2022 3:03PM,242136,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
3,Jul 18 2022 3:01PM,242135,12,HealthCaps LLC,1.0,NaN,NaN
4,Jul 18 2022 2:49PM,242133,10,"ClearSpec, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 18 2022 3:10PM,242096,10,Eywa Pharma Inc.,13.0,0.0,0.0
1,Jul 18 2022 3:05PM,242137,21,"NBTY Global, Inc.",1.0,0.0,0.0
2,Jul 18 2022 3:03PM,242136,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
3,Jul 18 2022 3:01PM,242135,12,HealthCaps LLC,1.0,0.0,0.0
4,Jul 18 2022 2:49PM,242133,10,"ClearSpec, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4357406,76.0,60.0,0.0
102,242008,1.0,0.0,0.0
12,242135,1.0,0.0,0.0
15,484119,6.0,10.0,10.0
19,968514,17.0,9.0,0.0
20,1936248,85.0,23.0,2.0
21,3631537,13.0,2.0,0.0
22,484232,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4357406,76.0,60.0,0.0
1,102,242008,1.0,0.0,0.0
2,12,242135,1.0,0.0,0.0
3,15,484119,6.0,10.0,10.0
4,19,968514,17.0,9.0,0.0
5,20,1936248,85.0,23.0,2.0
6,21,3631537,13.0,2.0,0.0
7,22,484232,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,76.0,60.0,0.0
1,102,1.0,0.0,0.0
2,12,1.0,0.0,0.0
3,15,6.0,10.0,10.0
4,19,17.0,9.0,0.0
5,20,85.0,23.0,2.0
6,21,13.0,2.0,0.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,76.0
1,102,Released,1.0
2,12,Released,1.0
3,15,Released,6.0
4,19,Released,17.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,0.0,10.0,0.0,2.0,0.0,0.0
Executing,60.0,0.0,0.0,10.0,9.0,23.0,2.0,0.0
Released,76.0,1.0,1.0,6.0,17.0,85.0,13.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,19,20,21,22
0,Completed,0.0,0.0,0.0,10.0,0.0,2.0,0.0,0.0
1,Executing,60.0,0.0,0.0,10.0,9.0,23.0,2.0,0.0
2,Released,76.0,1.0,1.0,6.0,17.0,85.0,13.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,19,20,21,22
0,Completed,0.0,0.0,0.0,10.0,0.0,2.0,0.0,0.0
1,Executing,60.0,0.0,0.0,10.0,9.0,23.0,2.0,0.0
2,Released,76.0,1.0,1.0,6.0,17.0,85.0,13.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()